In [1]:
! pip install numpy >> /dev/null
! pip install pandas >> /dev/null
! pip install opencv-python >> /dev/null
! pip install scipy >> /dev/null
! pip install torch >> /dev/null
! pip install matplotlib >> /dev/null
! pip install git+https://github.com/NVlabs/nvdiffrast >> /dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast /tmp/pip-req-build-wd91nwqj


In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import glob
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.spatial.transform import Rotation as R

In [3]:
INPUT_DIR = "data" # replace with your sequence
ORIGINAL_VIDEONAME = "output/orig.mp4"
UNDISTORTED_VIDEONAME = "output/undist.mp4"
RS_REMOVED_VIDEONAME = "output/nors.mp4"
STABILIZED_VIDEONAME = "output/stable_nors.mp4"
COMPARISON_VIDEONAME = "output/compar.mp4"

# camera params
FPS = 20

# camera intrinsics
# K = [739.1654756101043, 739.1438452683457, 625.826167006398, 517.3370973594253]
K = np.array([[739.1654756101043, 0.0, 625.826167006398],
              [0.0, 739.1438452683457, 517.3370973594253],
              [0.0, 0.0, 1.0]])

# output camera intrinsics
K_new = K # pominyatu potim



# distortion params
D = np.array([0.019327620961435945, 0.006784242994724914, -0.008658628531456217, 0.0051893686731546585])



T_cam_imu = np.array([[-0.0027687291789002095, -0.9999872674970001, 0.004218884048773266, -0.05688384754602901],
                       [-0.9999356528558058, 0.002814949729190873, 0.010989367855138411, 0.007618902284014501],
                       [-0.011001103879489809, -0.004188185992194848, -0.9999307150055582, -0.042436390295094266],
                       [0.0, 0.0, 0.0, 1.0]])

R_cam_imu = T_cam_imu[:3, :3]


dt_rs = 29.47 * 10 ** -6  # in seconds

# frame dimensions
W, H = 1280, 1024

newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(
    K_new, D, (W, H),
    alpha=0.0,   # 0 = crop black regions, 1 = keep all pixels
    newImgSize=(W, H)
)

In [4]:
def frames_to_video(frames, output_video, fps=20, isColor=False):
    """
    Create a video from a list of frames.

    :param frames: list of frames
    :param output_video: output video file name (e.g., 'output.mp4')
    :param fps: frames per second (default=20)
    """
    if len(frames) == 0:
        raise ValueError("No frames provided")

    # Use the first frame to get dimensions
    height, width, *channels = frames[0].shape

    # Define video writer (using mp4v codec)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, fps, (width, height), isColor=isColor)

    for frame in tqdm(frames, desc="Writing frames to video: "):
        video.write(frame)
        

frames_paths = sorted(glob.glob(os.path.join(INPUT_DIR, "cam1/data/*.png")))
print(f"Number of frames in the video: {len(frames_paths)}")

frames = [cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE) for frame_path in frames_paths]
frames_to_video(frames, ORIGINAL_VIDEONAME, FPS)

Number of frames in the video: 761


Writing frames to video: 100%|██████████| 761/761 [00:04<00:00, 183.17it/s]


In [5]:
import os
data = []
imgs_folder = INPUT_DIR + "/cam1" + "/data"
for f in os.listdir(imgs_folder):
    if os.path.isfile(os.path.join(imgs_folder, f)):
        base, ext = os.path.splitext(f)
        fid = int(base)
        data.append((fid, f))

imgs_data = pd.DataFrame(data, columns=["timestamp", "path"]).sort_values("timestamp").reset_index(drop=True)


### Homework 2

Given input video from camera with rolling shutter, implement and apply the following algorithms:

    - radial distortion correction (use cv2.fisheye module)
    - rolling shutter correction
    - video stabilization

Expected output of the homework:

    - Jupyter notebook with working code that we can run in real-time
    - plots of camera orientations in time for unstabilized and stabilized motion [plot individual x/y/z components in axis-angle representation]
    - plots of input/output meshgrids for the video frame, where angular velocity has the biggest norm
    - 4 videos stacked together with their frames side-by-side:
        - original
        - undistorted 
        - undistorted + rolling shutter corrected
        - undistorted + rolling shutter corrected + stabilized

Note:
Dataset used in homework provides two sources of camera orientations: IMU (Gyro) measurements of angular velocity and Motion Capture direct measurements of orientation. You should use IMU (Gyro) measurements and implement angular velocity integration. You can use Motion Capture direct measurements of orientation to verify that your integration is implemented correctly.

Rolling shutter correction algorithm is based on this paper: [Digital Video Stabilization and Rolling Shutter Correction using Gyroscopes](https://graphics.stanford.edu/papers/stabilization/).

Dataset: [Rolling-Shutter Visual-Inertial Odometry Dataset](https://cvg.cit.tum.de/data/datasets/rolling-shutter-dataset)

Calibrated camera params can be found [here](https://cdn3.vision.in.tum.de/rolling/calibration/camchain-calibration-equidistant4_camimu_dataset-calib-imu1.yaml)

![Camera Set-up](https://cvg.cit.tum.de/_media/data/datasets/rolling-shutter-dataset/sensor_axes_rgb.jpg?w=800&tok=fa3d4a)

## Preprocessing IMU data

In [6]:
imu_data = pd.read_csv(f"{INPUT_DIR}/imu0/data.csv", header=None, dtype=np.float64, skiprows=1)
imu_data.columns = ["timestamp", "w_x", "w_y", "w_z", "a_x", "a_y", "a_z"]
gyro_data = imu_data[["timestamp", "w_x", "w_y", "w_z"]].to_numpy()
accel_data = imu_data[["timestamp", "a_x", "a_y", "a_z"]].to_numpy()

In [7]:
gyro_timestamps_sec = gyro_data[:, 0] * 1e-9
gyro_start = gyro_timestamps_sec[0]
gyro_end = gyro_timestamps_sec[-1]
gyro_start_time = gyro_start
gyro_timestamps_sec -= gyro_start_time

In [8]:
def integrate_gyro(gyro):
    dt = np.diff(gyro[:, 0]) * 1e-9
    omega = gyro[:, 1:4]
    rotations = []

    R_curr = R.from_quat([1,0,0,0])
    max_rot = 0
    max_i = 0

    for i, (w, dt_) in enumerate(zip(omega, dt)):
        
        if np.linalg.norm(w) > max_rot:
            max_rot = np.linalg.norm(w)
            max_i = i

        dR = R.from_rotvec(w * dt_)
        R_curr = dR * R_curr
        rotations.append(R_curr.as_matrix())

    return np.array(rotations), max_i * dt[0]


rotations, i = integrate_gyro(gyro_data)
i

np.float64(33.763955200000005)

## Undistort frames

In [9]:
undistorted_frames = [cv2.fisheye.undistortImage(image, K, D, None, K) for image in tqdm(frames)]

100%|██████████| 761/761 [00:16<00:00, 44.83it/s]


In [10]:
frames_to_video(undistorted_frames, UNDISTORTED_VIDEONAME, FPS)

Writing frames to video: 100%|██████████| 761/761 [00:04<00:00, 183.82it/s]


## Rolling shutter correction

In [11]:
ts = dt_rs * H

In [12]:
from scipy.ndimage import gaussian_filter1d

def gaussian_low_pass(rotations, sigma=1):
    rot_objects = R.from_matrix(rotations)
    quats = rot_objects.as_quat()  
    smoothed_quats = np.zeros_like(quats)
    for i in range(4):
        smoothed_quats[:, i] = gaussian_filter1d(quats[:, i], sigma=sigma, mode='nearest')
    norms = np.linalg.norm(smoothed_quats, axis=1, keepdims=True)
    smoothed_quats = smoothed_quats / norms
    
    smoothed_rotations = R.from_quat(smoothed_quats).as_matrix()
    
    return smoothed_rotations

In [ ]:
from meshwarp import MeshWarper
meshwarper = MeshWarper()

vert_splits = 10
rotations_prime = gaussian_low_pass(rotations, sigma=15)
rotations_cam_all = np.array([R_cam_imu @ R @ R_cam_imu.T for R in rotations])
rotations_prime_cam_all = np.array([R_cam_imu @ R @ R_cam_imu.T for R in rotations_prime])

CUDA device available. Using cuda:0 for warping.


In [32]:
def compensate_rs(frame, rotations_cam_all, rotations_prime_cam_all, K, fps, ts, gyro_timestamps_sec, vert_splits=10, i=0, debug=False):
    h, w = frame.shape[:2]
    grid_dst, faces = meshwarper.build_meshgrid(w, h, vert_splits, vert_splits)
    grid_src = grid_dst.copy()
    grid_cols = vert_splits
    verts_per_row = grid_cols + 1 
    
    t_i = i / fps
    idx_i = np.searchsorted(gyro_timestamps_sec, t_i)
    idx_i = np.clip(idx_i, 0, len(rotations_prime_cam_all) - 1)

    if debug:
        print(f">>> Frame {i=} Initial time: {t_i=}, Gyroscope time: {gyro_timestamps_sec[idx_i]}")

    R_prime_ti = rotations_prime_cam_all[idx_i]
    
    for row_idx in range(vert_splits+1):
        y = row_idx * h / vert_splits
        t_row = t_i + ts * y / h
        
        idx = np.searchsorted(gyro_timestamps_sec, t_row)

        idx = np.clip(idx, 0, len(rotations_cam_all) - 1)

        if debug:
            print(f"----- Row {y=} Row time: {t_row=}, Gyroscope time: {gyro_timestamps_sec[idx]}, idx: {idx=}")

        R_t_row = rotations_cam_all[idx]
        
        W = K @ R_prime_ti @ R_t_row.T @ np.linalg.inv(K)
        start_idx = row_idx * verts_per_row
        end_idx = start_idx + verts_per_row
        
        points_h = np.concatenate([grid_dst[start_idx:end_idx], 
                                   np.ones((verts_per_row, 1)) ], axis=1)
        warped_h = (W @ points_h.T).T
        grid_src[start_idx:end_idx] = warped_h[:, :2] / warped_h[:, 2:3]

    if debug:
        fig, axes = plt.subplots(ncols=2)
        axes[0].imshow(meshwarper.draw_meshgrid(grid_src, faces, w, h))
        axes[1].imshow(meshwarper.draw_meshgrid(grid_dst, faces, w, h))
        plt.show()

    corrected = meshwarper.warp_grid(frame, grid_src, grid_dst, faces, (w, h))
    return corrected


### Output Comparison

In [ ]:
def crop_image(image, crop):
    h, w = image.shape[:2]
    return image[crop:h-crop,crop:w-crop]

def create_comparison_video(original_frames, undistorted_frames, compensated_frames, stabilized_frames, output_video, fps=20, isColor=False, crop=50):
    """
    Create a video from a list of frames.

    :param frames: list of frames
    :param output_video: output video file name (e.g., 'output.mp4')
    :param fps: frames per second (default=20)
    """
    frames = zip(original_frames, undistorted_frames, compensated_frames, stabilized_frames)
    if len(original_frames) == 0:
        raise ValueError("No frames provided")
    
    # Use the first frame to get dimensions
    height, width, *channels = undistorted_frames[0].shape
    height -= 2 * crop
    width -= 2 * crop

    # Define video writer (using mp4v codec)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video, fourcc, fps, (3*width, height), isColor=isColor)

    for (original, undistorted, compensated, stabilized) in tqdm(frames, desc="Writing frames to video: "):
        frame = np.ones((height, 3*width), dtype=np.uint8)

        frame[:, :width] = crop_image(undistorted, crop)
        frame[:, width:2*width] = crop_image(compensated, crop)
        frame[:, 2*width:] = crop_image(stabilized, crop)

        video.write(frame)


Writing frames to video: : 761it [00:12, 60.76it/s]


In [ ]:

compensated_frames = []
stabilized_frames = []


for i, frame in tqdm(enumerate(undistorted_frames)):
    compensated_frame = compensate_rs(
        undistorted_frames[i], 
        rotations_cam_all,
        rotations_cam_all,
        newCameraMatrix, 
        FPS, 
        ts,
        gyro_timestamps_sec,
        vert_splits,
        i=i
    )
    compensated_frames.append(compensated_frame)

    stabilized_frame = compensate_rs(
        undistorted_frames[i], 
        rotations_cam_all,
        rotations_prime_cam_all,
        newCameraMatrix, 
        FPS, 
        ts,
        gyro_timestamps_sec,
        vert_splits,
        i=i
    )
    stabilized_frames.append(stabilized_frame)

create_comparison_video(
    original_frames=frames,
    undistorted_frames=undistorted_frames,
    compensated_frames=compensated_frames,
    stabilized_frames=stabilized_frames,
    output_video=COMPARISON_VIDEONAME
) 


CUDA device available. Using cuda:0 for warping.


761it [00:10, 74.00it/s]
Writing frames to video: : 761it [00:12, 62.33it/s]
